In [32]:
import pickle

import numpy as np

from datasets.colmap import Parser

with open('rick.pkl', 'rb') as f:
    inputs = pickle.load(f)


# {
#     "depth": depth,
#     "camera_id": camera_id,
#     "image_name": image_name,
#     "parser": parser,
# },

depth: np.ndarray = inputs['depth']
camera_id:str = inputs['camera_id']
image_name:str = inputs['image_name']
parser: Parser = inputs['parser']

# TODO: GO OVER ALL OF THIS CUZ ALL OF IT IS PROBABLY WRONG

In [33]:
cam2world = parser.camtoworlds[camera_id]
K = parser.Ks_dict[camera_id]
imsize = parser.imsize_dict[camera_id]

sfm_points = parser.points[parser.point_indices[image_name]]
world2cam = np.linalg.inv(cam2world)
P = K @ world2cam[:3]

sfm_points_camera = P @ np.vstack([sfm_points.T, np.ones(sfm_points.shape[0])])
sfm_points_camera = sfm_points_camera[:2] / sfm_points_camera[2]
valid_sfm_pt_indices = np.logical_and(
        np.logical_and(sfm_points_camera[0] >= 0, sfm_points_camera[0] < imsize[0]),
        np.logical_and(sfm_points_camera[1] >= 0, sfm_points_camera[1] < imsize[1]),
    )
valid_sfm_pt_indices = np.logical_and(valid_sfm_pt_indices, sfm_points[:, 2] > 0)
sfm_points_camera = sfm_points_camera[
    :, valid_sfm_pt_indices
]

depth_ratios = sfm_points[valid_sfm_pt_indices, 2] / depth[sfm_points_camera[1].astype(int), sfm_points_camera[0].astype(int)]
depth_scalar = np.mean(depth_ratios)
print(f"{depth_scalar=}, {np.std(depth_ratios)=}")

depth_scalar=0.07240226110038772, np.std(depth_ratios)=0.040816114113786765


In [34]:
np.mgrid[0 : imsize[1], 0 : imsize[0]].T.shape

(1297, 840, 2)

In [35]:
camera_grid = np.dstack(
    [np.mgrid[0 : imsize[1], 0 : imsize[0]].T, depth.T * depth_scalar]
)
xyz = (np.linalg.inv(K) @ camera_grid.reshape(-1, 3).T).T

xyz = cam2world @ np.vstack([xyz.T, np.ones(xyz.shape[0])])

xyz = xyz[:3].reshape(imsize[1], imsize[0], 3)

# xyz[2] = depth.reshape(-1)
# xyz[:2] *= xyz[2]
# xyz = np.vstack([xyz, np.ones(xyz.shape[1])])

In [36]:
import torch.nn


factors = xyz[np.round(sfm_points_camera[1]).astype(int), np.round(sfm_points_camera[0]).astype(int)] / sfm_points[valid_sfm_pt_indices]
